In [40]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

data = pd.read_csv("clean_weather.csv", index_col=0)
data = data.ffill()

In [41]:
T_CUT = 2000
V_CUT = 1000
PREDICTORS = ["tmax", "tmin", "rain"]
TARGET = "tmax_tomorrow"

scaler = StandardScaler()
predictors = data[PREDICTORS].to_numpy()
predictors = scaler.fit_transform(predictors)
target = data[[TARGET]].to_numpy()

train = predictors[:-T_CUT,:]
train_target = target[:-T_CUT,:]

valid = predictors[-T_CUT:-V_CUT,:]
valid_target = target[-T_CUT:-V_CUT,:]

test = predictors[-V_CUT:,:]
test_target = target[-V_CUT:,:]

In [43]:
def mse(actual, predicted):
    return np.mean((actual-predicted)**2)

def mse_grad(actual, predicted):
    return (predicted - actual)

# Shape

## Inputs

1,3

1,10
1,10
1,1

## Weights

3,10
10,10
10,1


In [44]:
import numpy as np

In [45]:
def init_layers(inputs):
    layers = []
    for i in range(1, len(inputs)):
        layers.append([
            np.random.rand(inputs[i-1], inputs[i]) / 5 - .1,
            np.ones((1,inputs[i]))
        ])
    return layers

In [46]:
def forward(batch, layers):
    hidden = [batch.copy()]
    for i in range(len(layers)):
        batch = np.matmul(batch, layers[i][0]) + layers[i][1]
        hidden.append(batch.copy())
        if i < len(layers) - 1:
            batch = np.maximum(batch, 0)
        
    return layers, batch, hidden

In [58]:
def backward(layers, hidden, grad, lr):
    for i in range(len(layers)-1, -1, -1):
        if i != len(layers) - 1:
            grad = np.multiply(grad, np.heaviside(hidden[i+1], 1))
        
        grad = grad.T
        w_grad = np.matmul(grad, hidden[i]).T
        b_grad = grad.T
        
        layers[i][0] -= (w_grad + layers[i][0] * .01) * lr
        layers[i][1] -= b_grad * lr
        
        grad = np.matmul(layers[i][0], grad).T
    return layers

In [ ]:
layer_conf = [3, 10,10,1]
lr = 5e-4
epochs=70

layers = init_layers(layer_conf)

for epoch in range(epochs+1):
    epoch_loss = 0

    for i, row in enumerate(train):
        row = row.copy().reshape((1,train.shape[1]))
        layers, pred, hidden = forward(row, layers)

        loss = mse_grad(target[i,0], pred[0,0])
        epoch_loss += loss ** 2

        layers = backward(layers, hidden, np.array(loss).reshape(1,1), lr/train.shape[0])
    
    
    if epoch % 10 == 0:
        _, valid_preds, _ = forward(valid, layers)
        
        print(f"Epoch: {epoch} Train MSE: {epoch_loss/train.shape[0]} Valid MSE: {mse(valid_preds,valid_target)}")

Epoch: 0 Train MSE: 4208.062739950562 Valid MSE: 4442.600112935855
Epoch: 10 Train MSE: 3356.73810993645 Valid MSE: 3499.0968597104834
Epoch: 20 Train MSE: 694.1162293868707 Valid MSE: 663.6533216590747
